In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
from pymatreader import read_mat
from scipy import sparse
import numpy as np
import os
from datetime import datetime, timedelta
import pandas
import cv2
import json
import pandas as pd
from amftrack.pipeline.paths.directory import *
from amftrack.util import *
import pickle
from joblib import Parallel, delayed
from datetime import datetime
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.post_processing.time_plate import *

In [2]:
directory_project

'/projects/0/einf914/data/'

In [3]:
directory = directory_project
# directory = '/projects/0/einf914/agg/'
update_analysis_info(directory)
analysis_info = get_analysis_info(directory)

In [4]:
select = analysis_info.loc[analysis_info['Plate']!=22]
# analysis_info['Plate']=analysis_info['Plate'].fillna(758)

In [6]:
analysis_info['Plate']=analysis_info['Plate'].fillna(758)

In [8]:
select = analysis_info.loc[analysis_info['Plate'].isin([94])]

In [17]:
select = analysis_info.loc[analysis_info['Plate']==792]

In [ ]:
plates = set(select_v7['Plate'].values)
plates
selection = []
for plate in plates:
    select_folder = np.min(select_v7.loc[(select_v7['Plate']==plate)]['folder_analysis'])
    selection.append(select_folder)
select = select_v7.loc[(select_v7['folder_analysis'].isin(selection))]

In [ ]:
from shutil import copyfile
for index,row in select.iterrows():
    plate = row['Plate']
    folder_target = row['folder_analysis']
    if len(select.loc[select['Plate']==plate]['folder_analysis'])>0:
        folder_origin = select.loc[select['Plate']==plate]['folder_analysis'].values[0]
        src = f'{directory_project}{folder_origin}/orthog.npy'
        dst = f'{directory_project}{folder_target}/orthog.npy'
        if os.path.isfile(src):
            print(dst)
            # copyfile(src, dst)
        else:
            print(plate)

In [27]:
time = '10:00'
directory = directory
list_f = [num_hypha,prop_lost_tracks_junction,prop_lost_tracks_tips,prop_lost_tracks_junction,prop_lost_tracks_tips,prop_inconsistent_root,number_of_timepoints_withing_boundaries,number_of_timepoints]
list_args= [None,[1],[1],[10],[10]]+[[]]*len(list_f)
overwrite = True
num_parallel = 5
run_parallel_post('global_plate_post_process.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'global_plate_post_process',cpus = 32,name_job = 'glob_plate')

Submitted batch job 195252


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [11]:
time = '10:00'
directory = directory
list_f = []
list_args= [[]]*len(list_f)
overwrite = True
num_parallel = 5
run_parallel_post('exp_plot.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'global_plate_post_process',cpus = 128,name_job = 'glob_plate')

Submitted batch job 195518


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [12]:
time = '20:00'
directory = directory
list_f = []
list_args= [[]]*len(list_f)
overwrite = True
num_parallel = 5
run_parallel_post('make_small_exp.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'global_plate_post_process',cpus = 32,name_job = 'glob_plate')

Submitted batch job 195536


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [ ]:
frames = []
for index, row in select.iterrows():
    ser = pd.read_json(f'{directory}{row["path_global_plate_info"]}',convert_dates=True,typ='series')
    frame = ser.to_frame(index).transpose()
    frames.append(frame)
global_plate_infos = pd.concat(frames)
global_plate_infos

In [15]:
time = '20:00'
directory = directory
list_f = [get_num_trunks,get_length,get_area, get_area_separate_connected_components,get_num_tips,get_num_nodes,get_area_study_zone,get_num_tips_study_zone,get_num_nodes_study_zone,get_length_study_zone,is_out_study]
# list_f = [get_num_trunks]

list_args= [{}]*len(list_f)
overwrite = False
num_parallel = 5
run_parallel_post('time_plate_post_process.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'time_plate_post_process',cpus = 32,name_job = 'time_plate')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 196680


In [ ]:
frames = []
for index, row in select.iterrows():
    if os.path.isfile(f'{directory}{row["path_time_plate_info"]}'):
        frame = pd.read_json(f'{directory}{row["path_time_plate_info"]}',convert_dates=True).transpose()
        frame.index.name = 't'
        frame.reset_index(inplace=True)
        frames.append(frame)
time_plate_infos = pd.concat(frames,ignore_index=True)
time_plate_infos.loc[time_plate_infos['Plate']==91]

In [6]:
time = '3:00:00'
directory = directory
# list_f = [get_width,get_tot_length_C,get_tot_growth_C]
# list_f = [get_timestep_anastomosis]
# list_f = [get_tot_length_pp,get_tot_growth_pp,get_timestep_stop_growth,get_time_stop_growth,get_time_init_growth,get_mean_speed_growth,get_stop_track,get_timestep_anastomosis,get_timestep_biological_stop_growth]
# list_f = [get_num_branch]
# list_f = [gets_out_of_ROI]
list_args= [{}]*len(list_f)
overwrite = False
num_parallel = 2
run_parallel_post('global_hypha_post_process.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'global_hypha_post_process',cpus = 32,name_job = 'glob_hypha')

Submitted batch job 205332
Submitted batch job 205333


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [ ]:
frames = []
for index, row in select.iterrows():
    if os.path.isfile(f'{directory}{row["path_global_hypha_info"]}'):
        frame = pd.read_json(f'{directory}{row["path_global_hypha_info"]}',convert_dates=True).transpose()
        frame.index.name = 'hypha'
        frame.reset_index(inplace=True)
        frames.append(frame)
global_hypha_info = pd.concat(frames,ignore_index=True)
# time_plate_infos.loc[time_plate_infos['Plate']==91]

In [5]:

list_f = [get_distance_final_pos,get_timedelta,get_time_since_start,get_speed,get_timestep,get_timestep_init,get_time_init,get_degree,get_width_tip_edge,get_width_root_edge,get_width_average,has_reached_final_pos,in_ROI]
# list_f = [local_density,local_density,local_density]
list_f = [get_time_since_begin_exp]
# list_f = [local_density]
list_args= [{}]*len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = False
num_parallel = 25
time = '5:00'
for index, row in select.iterrows():
    folder = row['folder_analysis']
    path_time_plate_info = row['path_time_plate_info']
    plate = row['Plate']
    num_cpus = 32
    if os.path.isfile(f'{directory}{path_time_plate_info}'):
        whole_plate_info = pd.read_json(f'{directory}{path_time_plate_info}',
       convert_dates=True).transpose()
        whole_plate_info.index.name = 't'
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post('time_hypha_post_process.py', list_f,list_args,[directory,overwrite],whole_plate_info, num_parallel, time,'time_hypha_post_process',cpus = num_cpus,name_job='time_hypha')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 219433
Submitted batch job 219434
Submitted batch job 219435
Submitted batch job 219436


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 219437
Submitted batch job 219438
Submitted batch job 219439
Submitted batch job 219440
Submitted batch job 219441


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 219474
Submitted batch job 219475
Submitted batch job 219476
Submitted batch job 219477
Submitted batch job 219478


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 219479
Submitted batch job 219480
Submitted batch job 219481
Submitted batch job 219482
Submitted batch job 219483
Submitted batch job 219484
Submitted batch job 219485


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [ ]:
frames = []
for index, row in select.iterrows():
    folder = row['folder_analysis']
    path_time_plate_info = row['path_time_plate_info']
    plate = row['Plate']
    if os.path.isfile(f'{directory}{path_time_plate_info}'):
        time_plate_info = pd.read_json(f'{directory}{path_time_plate_info}',
       convert_dates=True).transpose()
        time_plate_info.index.name = 't'
        time_plate_info.reset_index(inplace=True)
        for indexo, rowt in time_plate_info.iterrows():
            t = rowt['t']
            path_hyph_info_t = f'{directory}{folder}/time_hypha_info/hyph_info_{t}.json'
            if os.path.isfile(path_hyph_info_t):
                hyph_info_t = pd.read_json(path_hyph_info_t,
       convert_dates=True).transpose()
                hyph_info_t['Plate']=rowt['Plate']
                hyph_info_t['folder']=folder
                hyph_info_t['t']=rowt['t']
                frames.append(hyph_info_t)
result = pd.concat(frames)

In [ ]:
result